# Статистика наизусть: A/B тестирование ч.1

Давайте сегодня поговорим о знаменитом A/B тестировании. Классическим примером является случайный показ пользователям двух разных баннеров и определение того, какой из этих двух баннеров эффективнее. Какой критерий эффективности баннера? Конечно же клики! Т.е. пользователь может кликнуть на баннер или не кликнуть. Представим, что у нас есть баннер A и баннер B и слудующая ситуция:

In [4]:
import pandas as pd
import numpy as np

In [6]:
case1 = pd.DataFrame([[1000,990],[1000,10]],
                     index=['A','B'],
                     columns=['Показы','Клики'],)
case1

,Показы,Клики
A,1000,990
B,1000,10


Очевидно, что баннер A работает лучше. Но если мы имеем такую ситуацию:

In [7]:
case2 = pd.DataFrame([[1000,200],[1000,180]],
                     index=['A','B'],
                     columns=['Показы','Клики'],)
case2

,Показы,Клики
A,1000,200
B,1000,180


Можем ли мы заявлять, что баннер A рабоет лучше? Здесь нам на помощь приходит статистика! Давайте вместе во всем разберемся. Похоже, что придется проверять какую-то гипотезу. А именно гипотезу о разности долей. Делается это следующим образом. При больших выборках биноминальное распределение (а у нас именно такое в данном случаее) может быть апроксимировано с помощью нормального. Чтобы использовать нормальное распределение нам нужно знать среднее значение или стандартное отклонение. Эти параметры получаеются следующим образом:

$$ \mu = p $$

$$ \sigma = \sqrt{\frac{p*(1-p)}{n}}$$

Значение p - это доля успехов, т.е. количество кликов деленное на количество показов баннера. Далее сформулируем основную и альтернативную гипотезы:

H0: p_A - p_B = 0
<br>H1: p_A - p_B != 0 </br>

Основная гипотеза - доли двух выборок не отличаются, альтернативная - доли отличаются. Апроксимируя доли выборок как нормальные переменные, мы так же предпологаем, что их разница будет иметь нормальное распределение со следующими средним и  стандартным отклонением:

$$ \mu_{A-B} = p_{B}-p_{A} $$

$$ \sigma_{A-B} = \sqrt{\sigma_{A}^2 + \sigma_{B}^2}$$

Далее остается только рассчитать z-статистику:

$$ z = \frac{\mu_{A-B}}{\sigma_{A-B}} $$

А теперь давайте оденем всё это в любимый Питон и произведем расчеты.

In [15]:
import math
from __future__ import division
def estimated_parameters(shown, clicked):
    p = clicked / shown
    sigma = math.sqrt(p * (1 - p) / shown)
    return p, sigma

def a_b_test_statistic(shown_A, clicked_A, shown_B, clicked_B):
    p_A, sigma_A = estimated_parameters(shown_A, clicked_A)
    p_B, sigma_B = estimated_parameters(shown_B, clicked_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

Рассчитаем z-статистику. Для наглядности я выведу еще раз выведу таблицу с исходными данными.

In [25]:
case2

,Показы,Клики
A,1000,200
B,1000,180


In [17]:
z = a_b_test_statistic(1000,200,1000,180)
z

-1.1403464899034472

А что мы делаем с z-статистикой? Правильно - рассчитываем p-value и делаем вывод отклонять или не отклонять основную гипотезу. Я возьму функцию нормального распределения из scipy. Нужно помнить, что эта функция считает одностороннее значение p, а в нашем случае гипотеза двухсторонняя, т.к. записана со знаком равенства (H0: p_A - p_B = 0 ). Вывод функции мы будем умножать на 2.

In [22]:
from scipy.stats import norm

In [24]:
norm.cdf(z)*2

0.25414197654223603

Значение p получилось достаточно большим, а значит, что в данном случае мы не можем отклонить основную гипотезу о равенстве двух долей. Интересно, чтобы получилось бы если у нас были те же самые доли, но при большем количестве испытаний? Давайте проверим. Чтобы не хардкодить значения, я возьму их из дата-фрейма следующим образом:

In [29]:
case2_values = case2.values.ravel()
case2_values

array([1000,  200, 1000,  180])

Теперь увеличим их в два раза и рассчитаем p-value.

In [31]:
z = a_b_test_statistic(*case2_values*2)
z

-1.6126934718260086

In [32]:
norm.cdf(z)*2

0.10681111954161675

p уменьшилось! Это значит, что чем больше экспериментов мы проводим, тем более статистически значимой становится разница. Обычно нулевую гипотезу отвергают при значениях p меньше 0.05. Теперь вы знаете, как проводить A/B тестирование!

Справедливости ради стоит отметить, что в данном случае мы использовали нормальное распределение не зная стандартного отклонинея совокупности - мы лишь апроксимировали его. В этой ситуации, по-хорошему мы должны были использовать распределение Стьюдента, но дело в том, что при больших выборках (а у нас была достаточно большая выборка) z-статистика будет совсем не сильно отличаться от t-статистики.

Во второй части статьи про A/B тестирование я расскажу, как проводить A/B тестирование с помощью статистики хи-квадрат, а так же подробнее расскажу о самом хи-квадрате.